In [20]:
# Construction of dataset

import os, itertools, time, pickle, sys, glob, requests
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import matplotlib.pyplot as plt
from orderedset import OrderedSet
from copy import deepcopy
%matplotlib inline  

In [21]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.construct_mapping_dict()
        
        self.parents_dict = {}
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()        
    
    def construct_mapping_dict(self):
        self.mapping_dict = {self.extract_ID(el, False): self.get_child_node(el, "rdfs:label")[0].firstChild.nodeValue for el in self.root.getElementsByTagName("owl:Class") if self.get_child_node(el, "rdfs:label")}
        self.mapping_dict_inv = {self.mapping_dict[key]: key for key in self.mapping_dict}
        return
        
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self, rootpath=False):
        subclasses = self.get_subclasses(rootpath=False)
        return [(b,a,c,d) for (a,b,c,d) in subclasses]
    
    def parse_triples(self, union_flag=0, subclass_of=True, rootpath=False):
        obj_props = [(prop, "Object Property") for prop in self.object_properties]
        data_props = [(prop, "Datatype Property") for prop in self.data_properties]
        props = obj_props + data_props
        all_triples = []
        for prop, prop_type in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop), prop_type) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop), prop_type))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples(rootpath))
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True, rootpath=False):
        return self.parse_triples(union_flag, subclass_of, rootpath)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode, "subclass_of", "Subclass"))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    restriction = el.getElementsByTagName("owl:Restriction")
                    if not restriction:
                        continue
                    prop = self.get_child_node(restriction[0], "owl:onProperty")
                    some_vals = self.get_child_node(restriction[0], "owl:someValuesFrom")
                    
                    if not prop or not some_vals:
                        continue
#                     print(self.extract_ID(el), "**", self.extract_ID(some_vals[0]), "**", self.extract_ID(prop[0]))
                    try:
                        if self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                        else:
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                    except:
                        try:
                            if not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                            elif not self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                class_vals = self.get_child_node(some_vals[0], "owl:Class")
                                subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                        except Exception as e:
                            print (e)
                            continue
                else:
                    if self.extract_ID(level1_class[0]):
                        subclass_pairs.append((level1_class[0], el.parentNode, "subclass_of", "Subclass"))
                    else:
#                         level2classes = level1_class[0].getElementsByTagName("owl:Class")
#                         subclass_pairs.extend([(elem, el.parentNode, "subclass_of", "Subclass") for elem in level2classes if self.extract_ID(elem)])
                        continue
        return subclass_pairs
        
    def get_subclasses(self, rootpath=False):
        subclasses = [(self.extract_ID(a, not rootpath), self.extract_ID(b, not rootpath), c, d) for (a,b,c,d) in self.subclasses]
        self.parents_dict = {}
        for (a,b,c,d) in subclasses:
            if c == "subclass_of" and a!="Thing" and b!="Thing":
                if b not in self.parents_dict:
                    self.parents_dict[b] = [a]
                else:
                    self.parents_dict[b].append(a)
        return [el for el in subclasses if el[0] and el[1] and el[2] and el[0]!="Thing" and el[1]!="Thing"]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element, check_coded = True):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        element_id = element_id.split("#")[-1]
        if len(list(filter(str.isdigit, element_id))) >= 3 and "_" in element_id and check_coded:
            return self.mapping_dict[element_id]
        return element_id.replace("UNDEFINED_", "").replace("DO_", "")
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))


In [22]:
alignment_folder = "german_datasets/"

ontologies_in_alignment = []

# Load reference alignments 
def load_alignments(folder):
    gt = []
    for subdir in os.listdir(folder):
        path = folder + subdir + "/"
        path += [l for l in os.listdir(path) if l.endswith(".txt")][0]
        mappings = [content.strip() for content in open(path).read().split("--------------------------------------------------------") if content.strip()]
        for mapping in mappings:
            src = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Source: ")][0]
            targ = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Target: ")][0]
            ontologies_in_alignment.append((folder + subdir + "/"+src, folder + subdir + "/"+targ))
            src = src.rsplit(".",1)[0].replace(".", "_").lower()
            targ = targ.rsplit(".",1)[0].replace(".", "_").lower()
            lines = [["_".join(row.strip().split(":")[0].split(".")) for row in line.split("-",1)[1].strip().split("<->")]
                     for line in mapping.split("\n") if line.startswith(" -")]
            lines = [[src + "#" + line[0], targ + "#" + line[1]] for line in lines]
            gt.extend(lines)
    return gt

# Extracting USE embeddings

def extractUSEEmbeddings(words):
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3?tf-hub-format=compressed")
    embeds = model(words)
    return embeds.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)


reference_alignments = load_alignments(alignment_folder)


In [23]:
# Combinatorial mapping generation
all_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology(l[0])
    ont2 = Ontology(l[1])
    
    ent1 = ont1.get_classes()
    ent2 = ont2.get_classes()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()

    mappings = list(itertools.product(ent1, ent2)) + list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))
    
    pre1 = l[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    pre2 = l[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    print (pre1, pre2)
    all_mappings.extend([(pre1 + "#" + el[0], pre2 + "#" + el[1]) for el in mappings])
    

data = {mapping: False for mapping in all_mappings}
reference_alignments = [tuple(alignment) for alignment in reference_alignments]
s = set(all_mappings)
for mapping in set(reference_alignments):
    if mapping in s:
        data[mapping] = True
    else:
        mapping = tuple([el.replace(",-", "_") for el in mapping])
        if mapping in s:
            data[mapping] = True
        else:
            print (mapping)

google_lebensmittel web_lebensmittel
dmoz_freizeit google_freizeit
dmoz google
dmoz web
dmoz yahoo_small
google web
google yahoo_small
web yahoo_small


In [11]:


def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

# Abbrevation resolution preprocessing
def parse(word):
    return " ".join(flatten([el.split("_") for el in camel_case_split(word)]))

abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_") if el])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_") if el])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings([parse(el) for el in keys])))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}


right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Getraenke_Wasser', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOEUTGW Essen_und
right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Getraenke_Spirituosen_Absinth', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOEUTGSA Essen_und
right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Getraenke_Cocktails', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOEUTGC Essen_und
right ('Top_World_Deutsch_Online-Shops_Fahrzeuge_Autos_Ersatzteile_und_Zubehoer_Ersatzteile', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOFAEUZE Ersatzteile_und
right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Regionale_Spezialitaeten_Europaeisch_Spanisch', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOEUTRSES Essen_und
right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Getraenke_Wein_Europaeisch', 'World_Deutsch_Online-Shops_Fahrzeug

right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Honig', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOEUTH Essen_und
right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Essig_und_Oel', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOEUTEUO Essen_und
right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Getraenke', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOEUTG Essen_und
right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Getraenke_Wein_Europaeisch_Deutsch_Mosel-Saar-Ruwer', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOEUTGWEDM Essen_und
right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Vegetarisch', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOEUTV Essen_und
right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Regionale_Spezialitaeten_Europaeisch_Oesterreichisch', 'World_Deutsch_Online-Shops_Fahrzeuge_Autos_EU-Importfahrzeuge') TWDOEUTR

right ('Top_World_Deutsch_Online-Shops_Schmuck_Modeschmuck_Perlen', 'Verzeichnis_Einkaufen-Sparen_Buecher_Musik-Filme_Musik_MP3-Files') TWDOSMP Modeschmuck_Perlen
right ('Top_World_Deutsch_Online-Shops_Fahrzeuge_Motorraeder_Pflegemittel', 'Verzeichnis_Einkaufen-Sparen_Buecher_Musik-Filme_Musik_MP3-Files') TWDOFMP Motorraeder_Pflegemittel
right ('Top_World_Deutsch_Online-Shops_Unterhaltung_Bild_und_Tontraeger_Video_DVD', 'Verzeichnis_Einkaufen-Sparen_Unterhaltungs-Kommunikationselektronik_Unterhaltungselektronik_Satelliten-Digital-TV') TWDOUBUTVD Tontraeger_Video
right ('Top_World_Deutsch_Online-Shops_Essen_und_Trinken_Vegetarisch', 'Verzeichnis_Einkaufen-Sparen_Unterhaltungs-Kommunikationselektronik_Unterhaltungselektronik_Satelliten-Digital-TV') TWDOEUTV Trinken_Vegetarisch
right ('Top_World_Deutsch_Online-Shops_Bekleidung_Unterwaesche_Damen', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') TWDOBUD Online-Shops_Bekleidung
right ('Top_World_Deutsch_Online-Shops_Kunsthandwerk_Glas

right ('Top_World_Deutsch_Online-Shops_Bekleidung_Sondergroessen', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') TWDOBS Online-Shops_Bekleidung
right ('Top_World_Deutsch_Online-Shops_Publikationen_Buecher_Religion_und_Spiritualitaet_Esoterik', 'Firmen_Kraftfahrzeuge_Ersatzteile-und-Zubehoer_US-Fahrzeuge') TWDOPBRUSE und_Spiritualitaet
right ('Top_World_Deutsch_Online-Shops_Bekleidung_Herren', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') TWDOBH Online-Shops_Bekleidung
right ('Top_World_Deutsch_Online-Shops_Bekleidung_Trachten_Europaeisch_Alpenlaendisch', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') TWDOBTEA Online-Shops_Bekleidung
right ('Top_World_Deutsch_Online-Shops_Bekleidung_Herren_Oberhemden', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') TWDOBHO Online-Shops_Bekleidung
right ('Top_World_Deutsch_Online-Shops_Bekleidung_Kostueme_Historisch_Mittelalterlich', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') TWDOBKHM Online-Shops_Bekleidung
right

right ('Top_World_Deutsch_Online-Shops_Computer_Hardware_Komplettsysteme', 'Firmen_Computer_Hardware_Beamer-und-Projektoren_HK-Computer-Systemhaus') TWDOCHK Hardware_Komplettsysteme
right ('Top_World_Deutsch_Online-Shops_Haus_und_Garten_Haushaltsbedarf_Haushaltsgeraete_Kaffeemaschinen', 'Firmen_Computer_Hardware_Beamer-und-Projektoren_HK-Computer-Systemhaus') TWDOHUGHHK Haushaltsgeraete_Kaffeemaschinen
right ('Top_World_Deutsch_Online-Shops_Bekleidung_Behinderte', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') TWDOBB Online-Shops_Bekleidung
right ('Top_World_Deutsch_Online-Shops_Religion_und_Spiritualitaet_Esoterik_Raeucherwerk__Duefte', 'Firmen_Kraftfahrzeuge_Ersatzteile-und-Zubehoer_US-Fahrzeuge') TWDORUSERD und_Spiritualitaet
right ('Top_World_Deutsch_Online-Shops_Schmuck_Edel_und_Schmucksteine_Opale', 'Firmen_Kraftfahrzeuge_Ersatzteile-und-Zubehoer_US-Fahrzeuge') TWDOSEUSO und_Schmucksteine
right ('Top_World_Deutsch_Online-Shops_Bekleidung_Kinder', 'Firmen_Koerperpflege_Hygi

right ('World_Deutsch_Online-Shops_Blumen', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOB Online-Shops_Blumen
right ('World_Deutsch_Online-Shops_Bekleidung_Accessoires_Krawatten', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOBAK Online-Shops_Bekleidung
right ('World_Deutsch_Online-Shops_Bekleidung_Schuhe', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOBS Online-Shops_Bekleidung
right ('World_Deutsch_Online-Shops_Bekleidung', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOB Online-Shops_Bekleidung
right ('World_Deutsch_Online-Shops_Bekleidung_Accessoires_Guertel-und-Hosentraeger', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOBAG Online-Shops_Bekleidung
right ('World_Deutsch_Online-Shops_Publikationen_Comics', 'Firmen_Computer_Hardware_PCs') WDOPC Publikationen_Comics
right ('World_Deutsch_Online-Shops_Bekleidung_Sondergroessen_Damen', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOBSD Online-Shops_Bekleidung
right ('Wor

right ('World_Deutsch_Online-Shops_Bekleidung_Unterwaesche_Socken-und-Struempfe_Abonnement', 'Firmen_Kraftfahrzeuge_Ersatzteile-und-Zubehoer_US-Fahrzeuge') WDOBUSA Unterwaesche_Socken-und-Struempfe
right ('World_Deutsch_Online-Shops_Bekleidung_Unterwaesche_Socken-und-Struempfe_Abonnement', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOBUSA Online-Shops_Bekleidung
right ('World_Deutsch_Online-Shops_Bekleidung_Sportswear', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOBS Online-Shops_Bekleidung
right ('World_Deutsch_Online-Shops_Schreibwaren-und-Buerobedarf_Materialien-und-Zubehoer_Stempel', 'Firmen_Telekommunikation_Handy-und-SMS_Mobilfunknetz-Anbieter_D2-Vodafone') WDOSMS Schreibwaren-und-Buerobedarf_Materialien-und-Zubehoer_Stempel
right ('World_Deutsch_Online-Shops_Schreibwaren-und-Buerobedarf_Materialien-und-Zubehoer_Stempel', 'Firmen_Telekommunikation_Handy-und-SMS_Geraete_Gebrauchte-Handys') WDOSMS Schreibwaren-und-Buerobedarf_Materialien-und-Zubehoer_Stempel

right ('World_Deutsch_Online-Shops_Unterhaltung_Bild-und-Tontraeger_Musik_Genres', 'Firmen_Musik_Labels_Bertelsmann-Music-Group-Entertainment-BMG') WDOUBMG Bild-und-Tontraeger_Musik_Genres
right ('World_Deutsch_Online-Shops_Computer_Hardware_Komponenten', 'Firmen_Computer_Hardware_Beamer-und-Projektoren_HK-Computer-Systemhaus') WDOCHK Hardware_Komponenten
right ('World_Deutsch_Online-Shops_Bekleidung_Kinder_Secondhand', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOBKS Online-Shops_Bekleidung
right ('World_Deutsch_Online-Shops_Bekleidung_Behinderte', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOBB Online-Shops_Bekleidung
right ('World_Deutsch_Online-Shops_Bekleidung_Accessoires_Schals-und-Tuecher', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOBAS Online-Shops_Bekleidung
right ('World_Deutsch_Online-Shops_Bekleidung_Schuhe_Sondergroessen', 'Firmen_Koerperpflege_Hygieneartikel-fuer-Frauen_OB') WDOBSS Online-Shops_Bekleidung
right ('World_Deutsch_Online-

In [12]:
extracted_elems = []
mapping_ont = {}
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology(ont_name)
    entities = ont.get_classes()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples())))
    ont_name = ont_name.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    print (ont_name)
    mapping_ont[ont_name] = ont
    extracted_elems.extend([ont_name + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))
inp = []
for word in extracted_elems:
    ont_name = word.split("#")[0]
    elem = word.split("#")[1]
    inp.append(parse(elem))

print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

extracted_elems = ["<UNK>"] + extracted_elems

embeds = np.array(extractUSEEmbeddings(inp))
embeds = np.array([np.zeros(embeds.shape[1],)] + list(embeds))
# embeds = np.array([np.zeros(512,)] + list(extractUSEEmbeddings(inp_spellchecked)))
embeddings = dict(zip(extracted_elems, embeds))


emb_vals = list(embeddings.values())
emb_indexer = {key: i for i, key in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: key for i, key in enumerate(list(embeddings.keys()))}


dmoz_freizeit
dmoz
yahoo_small
web_lebensmittel
web
google_freizeit
google
google_lebensmittel
Total number of extracted unique classes and properties from entire RA set:  3290


In [7]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology(ont)
    triples = ont_obj.get_triples(rootpath=True)
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    print (ont)
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append([e2])
            neighbours_dict[e2][2].append([e1])
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append([e2])
            neighbours_dict[e2][3].append([e1])
        elif d == "Subclass":
            neighbours_dict[e2][1].append([e1])
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
#         print ("Done for ", elem)
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].split(".")[0]

    
    for entity in neighbours_dict:
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     neighbours_dict = {elem: [key[:1] + sorted(list(set(key[1:]))) for key in neighbours_dict[elem]]
#                        for elem in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [[tuple([ont + "#" + node for node in path]) for path in nbr_type]
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

# neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}
max_types = np.max([len([nbr_type for nbr_type in elem if nbr_type]) for elem in neighbours_dicts.values()])
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)

# max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
# max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
                                for nbr_type in neighbours_dicts[elem]] for elem in neighbours_dicts}
neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
                                for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}
data_items = data.items()
data_shuffled_t = [elem for elem in data_items if elem[1]]
data_shuffled_f = [elem for elem in data_items if not elem[1]]
np.random.shuffle(data_shuffled_f)
data_shuffled_f = data_shuffled_f[:150000-len(data_shuffled_t)]

# indices = np.random.permutation(len(data_shuffled_t) + len(data_shuffled_f[:130000-len(data_shuffled_t)]))

data_shuffled = data_shuffled_t + data_shuffled_f
indices = np.random.permutation(len(data_shuffled))

# indices = np.random.permutation(len(data_shuffled))

data = OrderedDict(list(np.array(data_shuffled)[indices]))

ontologies_in_alignment_rev = [[el.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() for el in ont] for ont in ontologies_in_alignment]

# f = open("data_aml_uniqpath.pkl", "wb")
# pickle.dump([data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts_pathpadded, ontologies_in_alignment], f)
# # # # # neighbours_dicts

german_datasets/mapping webdirectory/google.owl
german_datasets/mapping webdirectory/dmoz.owl
german_datasets/mapping lebensmittel/web.Lebensmittel.owl
german_datasets/mapping lebensmittel/Google.Lebensmittel.owl
german_datasets/mapping freizeit/dmoz.Freizeit.owl
german_datasets/mapping freizeit/Google.Freizeit.owl
german_datasets/mapping webdirectory/yahoo.small.owl
german_datasets/mapping webdirectory/web.owl


NameError: name 'data' is not defined

In [18]:
data, emb_indexer, emb_indexer_inv, emb_vals, reference_alignments, _, ontologies_in_alignment_rev = pickle.load(open("../Input/data_german_dataset.pkl", "rb"))

In [19]:
f = open("Input/data_german_dataset_demarcated_bagofnbrs.pkl", "wb")
pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, reference_alignments, neighbours_dicts, ontologies_in_alignment_rev, max_types, max_paths, max_pathlen], f)


In [16]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology(ont)
    triples = ont_obj.get_triples(rootpath=True)
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    print (ont)
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append(e2)
            neighbours_dict[e2][2].append(e1)
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append(e2)
            neighbours_dict[e2][3].append(e1)
        elif d == "Subclass":
            neighbours_dict[e2][1].append(e1)
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
#         print ("Done for ", elem)
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].split(".")[0]

    for entity in neighbours_dict:
        neighbours_dict[entity][1] = [neighbours_dict[entity][1]]
        neighbours_dict[entity][2] = [neighbours_dict[entity][2]]
        neighbours_dict[entity][3] = [neighbours_dict[entity][3]]
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     neighbours_dict = {elem: [key[:1] + sorted(list(set(key[1:]))) for key in neighbours_dict[elem]]
#                        for elem in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [[tuple([ont + "#" + node for node in path]) for path in nbr_type]
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

# neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}
max_types = np.max([len([nbr_type for nbr_type in elem if nbr_type]) for elem in neighbours_dicts.values()])
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)

# max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
# max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
                                for nbr_type in neighbours_dicts[elem]] for elem in neighbours_dicts}
neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
                                for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}
data_items = data.items()
data_shuffled_t = [elem for elem in data_items if elem[1]]
data_shuffled_f = [elem for elem in data_items if not elem[1]]
np.random.shuffle(data_shuffled_f)
data_shuffled_f = data_shuffled_f[:150000-len(data_shuffled_t)]

# indices = np.random.permutation(len(data_shuffled_t) + len(data_shuffled_f[:130000-len(data_shuffled_t)]))

data_shuffled = data_shuffled_t + data_shuffled_f
indices = np.random.permutation(len(data_shuffled))

# indices = np.random.permutation(len(data_shuffled))

data = OrderedDict(list(np.array(data_shuffled)[indices]))

ontologies_in_alignment_rev = [[el.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() for el in ont] for ont in ontologies_in_alignment]

# f = open("data_aml_uniqpath.pkl", "wb")
# pickle.dump([data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts_pathpadded, ontologies_in_alignment], f)
# # # # # neighbours_dicts

german_datasets/mapping webdirectory/google.owl
german_datasets/mapping webdirectory/dmoz.owl
german_datasets/mapping lebensmittel/web.Lebensmittel.owl
german_datasets/mapping lebensmittel/Google.Lebensmittel.owl
german_datasets/mapping freizeit/dmoz.Freizeit.owl
german_datasets/mapping freizeit/Google.Freizeit.owl
german_datasets/mapping webdirectory/yahoo.small.owl
german_datasets/mapping webdirectory/web.owl


In [ ]:
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts2.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts2.values()]), axis=0)
neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
                                for nbr_type in neighbours_dicts2[elem]] for elem in neighbours_dicts2}
neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
                                for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}

np.array(list(neighbours_dicts_pathpadded.values())).shape

In [ ]:
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts1.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts1.values()]), axis=0)
neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
                                for nbr_type in neighbours_dicts1[elem]] for elem in neighbours_dicts1}
neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
                                for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}


In [17]:
max_paths, max_pathlen, max_types

(1, 49, 4)

In [ ]:
embedding_dim = np.array(emb_vals).shape[1]
        
name_embedding = nn.Embedding(len(emb_vals), embedding_dim)
name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
name_embedding.weight.requires_grad = False

cosine_sim_layer = nn.CosineSimilarity(dim=1)
output = nn.Linear(2*embedding_dim, 300)
max_pathlen = np.array(list(neighbours_dicts_pathpadded.values())).shape[3]
v = nn.Parameter(torch.DoubleTensor([1/(max_pathlen) for i in range(max_pathlen)]))

nodes = torch.randint(0, len(emb_vals), size=(10,2))
features= torch.randint(0, len(emb_vals), size=(10,2,4,22,6))

nodes = nodes.permute(1,0) # 2 * batch_size
features = features.permute(1,0,2,3,4) # 2 * batch_size * 4 * max_paths * max_pathlen
for i in range(2):
    node_emb = name_embedding(nodes[i]) # batch_size * 512
    feature_emb = name_embedding(features[i]) #  batch_size * 4 * max_paths * max_pathlen * 512

    path_weights = torch.sum(torch.sum(node_emb[:, None, None, None, :] * feature_emb, dim=-1), dim=-1)
    best_path_indices = torch.max(path_weights, dim=2)[1][(..., ) + (None, ) * 3]
    best_path_indices = best_path_indices.expand(-1, -1, -1, max_pathlen,  embedding_dim)
    best_path = torch.gather(feature_emb, 2, best_path_indices).squeeze(2) # batch_size * 4 * max_pathlen * 512
    # Another way: 
    # path_weights = masked_softmax(path_weights)
    # best_path = torch.sum(path_weights.unsqueeze(-1) * feature_emb, dim=2)

    node_weights = torch.sum(node_emb[:, None, None, :] * best_path, dim=-1).unsqueeze(-1)
    torch.matmul(v, node_weights * best_path)

In [ ]:
def masked_softmax(inp):
    inp = inp.double()
    mask = ((inp != 0).double() - 1) * 9999  # for -inf
    return (inp + mask).softmax(dim=-1)
a = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,22,6,512))

attended_path = torch.bmm(a.reshape(-1, 1, 512), b.reshape(-1, 512, 1))
attended_path = attended_path.reshape(-1, 4, 22, 6)
path_weights = masked_softmax(torch.sum(attended_path, dim=-1))
path_weights.shape
# best_path = torch.sum(path_weights[:, :, :, None, None] * a, dim=2)

In [ ]:
indices = d[(..., ) + (None, ) * 3].expand(-1, -1, -1, 6, 512)
e = torch.gather(b, 2, indices).squeeze(2)

f = torch.sum(a[:,None,None,:] *e,dim=-1).unsqueeze(-1)

g = (f*e)

h = torch.sum((v[None,None,:,None] * g), dim=2)

In [ ]:
c = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,1,6,512))
d = b * c


In [ ]:
e = torch.bmm(c.permute(0,1,3,4,2).reshape(-1, 22, 1), b.permute(0,1,3,4,2).reshape(-1, 1, 1)).squeeze(-1).reshape(-1,4,6,512,22).permute(0,1,4,2,3)


In [ ]:
fo

In [24]:
len(data)

3318178

In [ ]:
ctimes, dtimes = [], []
for i in range(100):
    a = torch.randn((10,4,22,6,512))
    b = torch.randn((10,4,1,6,512))
    t = time.time()
    c = a * b
    ctimes.append(time.time()-t)
    t = time.time()
    d = torch.bmm(c.permute(0,1,3,4,2).reshape(-1, 22, 1), b.permute(0,1,3,4,2).reshape(-1, 1, 1)).squeeze(-1).reshape(-1,4,6,512,22).permute(0,1,4,2,3)
    dtimes.append(time.time()-t)

In [ ]:
a = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,22,6,512))
c = a * torch.sum(b,dim=2).unsqueeze(2)

In [ ]:
np.array([(a[0][0][0] * b[0][0][i]).numpy() for i in range(22)]).shape

In [ ]:
# AML test
def is_test(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) in test_onto

results = []
all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))
for i in list(range(0, len(all_ont_pairs), 3)):
    test_onto = all_ont_pairs[i:i+3]
    for ont_pair in test_onto:
        a, b, c = ont_pair[0], ont_pair[1], ont_pair[0] + "-" + ont_pair[1]
        java_command = "java -jar AML_v3.1/AgreementMakerLight.jar -s conference_ontologies/" + a + ".owl" + \
                            " -t conference_ontologies/" + b + ".owl -o AML-test-results/" + c + ".rdf -a"
        process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    print (os.listdir("AML-test-results/"))
    pred_aml = load_alignments("AML-test-results/")
    pred_aml = [tuple([el.split("/")[-1] for el in key]) for key in pred_aml]
    tp = len([elem for elem in pred_aml if data[elem]])
    fn = len([key for key in gt_mappings if key not in set(pred_aml) and is_test(test_onto, key)])
    fp = len([elem for elem in pred_aml if not data[elem]])

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1score = 2 * precision * recall / (precision + recall)
    f2score = 5 * precision * recall / (4 * precision + recall)
    f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    print (precision, recall, f1score, f2score, f0_5score)
    
    metrics = [precision, recall, f1score, f2score, f0_5score]
    results.append(metrics)
    
    _ = [os.remove(f) for f in glob.glob('AML-test-results/*')]
    
print ("Final Results:", np.mean(results, axis=0))

In [ ]:
ontologies_in_alignment = [[el.split("/")[1].split(".")[0] for el in ont] for ont in ontologies_in_alignment][:-1] + [["human", "mouse"]]

In [ ]:
nn.Parameter(torch.DoubleTensor([1/(max_pathlen) for i in range(max_pathlen)])).shape

In [ ]:
f = open("data_unhas.pkl", "wb")
pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment], f)


In [ ]:
def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])
neighbours_dicts = {ont: {el: neighbours_dicts[ont][el][:int(sys.argv[1])] for el in neighbours_dicts[ont]
       if count_non_unk(neighbours_dicts[ont][el]) > int(sys.argv[2])} for ont in neighbours_dicts}

In [ ]:
def add1(elem):
    return elem + 1

v = np.random.random((3,2,3))
v, np.vectorize(add1)([[1,2,3],[1,2,3]])

In [ ]:
neighbours_dicts

In [ ]:
import requests

url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

# inp_spellchecked = []
for concept in inp[731:]:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word, response["corrections"][word][0])
        
        inp_spellchecked.append(resolved)
        print (concept, resolved)
    else:
        inp_spellchecked.append(concept)




In [ ]:
querystring = {"text": "technically Organised By"}
response = requests.request("GET", url, headers=headers, params=querystring)
response.json()

In [ ]:
inp_spellchecked[730], inp[731]

In [ ]:
fn_spellchecked, fp_spellchecked = [dict(el) for el in pickle.load(open("test_v2.pkl", "rb"))]
fn_baseline, fp_baseline = [dict(el) for el in pickle.load(open("test_best.pkl", "rb"))]
fn_unhas, fp_unhas = [dict(el) for el in pickle.load(open("test_unhas.pkl", "rb"))]
fn_resolved, fp_resolved = [dict(el) for el in pickle.load(open("test_resolved.pkl", "rb"))]

fn_dict, fp_dict = {}, {}
def create_comparison_file(file, idx):
    fn, fp = [dict(el) for el in pickle.load(open(file, "rb"))]
    
    for key in fn:
        if key in fn_dict:
            fn_dict[key][idx] = fn[key]
        else:
            fn_dict[key] = ["N/A" for i in range(4)]
            fn_dict[key][idx] = fn[key]
    
    for key in fp:
        if key in fp_dict:
            fp_dict[key][idx] = fp[key]
        else:
            fp_dict[key] = ["N/A" for i in range(4)]
            fp_dict[key][idx] = fp[key]
    

create_comparison_file("test_best.pkl", 0)
create_comparison_file("test_unhas.pkl", 1)
create_comparison_file("test_v2.pkl", 2)
create_comparison_file("test_resolved.pkl", 3)

open("fn - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fn_dict.items()]))
open("fp - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fp_dict.items()]))

In [ ]:
ontologies_in_alignment = pickle.load(open("data_path.pkl", "rb"))[-1]
ontologies_in_alignment

In [ ]:
d = {('confOf#Organization', 'sigkdd#Organizator'): (1,2,3,4),
 ('iasted#Document', 'sigkdd#Document'): (5,6,78,8)}
[[str(el) for el in flatten(el)] for el in d.items()]

In [ ]:
abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    inp_resolved.append(concept)
inp_resolved

In [ ]:
keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))


In [ ]:
cos_sim(*extractUSEEmbeddings(["Conference Banquet", "Dinner Banquet"]))

In [ ]:
fn

In [ ]:
scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)


In [ ]:
inp_case_handled = []
for concept in inp:
    final_list = []
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", concept):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    case_resolved = " ".join(final_list)
    inp_case_handled.append(case_resolved)
    
inp_case_handled

In [ ]:
Ontology("conference_ontologies/conference.owl").triples

In [ ]:
cos_sim2